# Tensor Methods Homework 3

### 1 Implementation of the HOSVD algorithm

Implement the HOSVD algorithm for the Tucker approximation of a given tensor $A \in \R^{n_1 \times \dots \times n_d}$ (with dimensionality $d \in \N$ and mode sizes $n_1, \dots, n_d \in \N$) with given ranks $r_1, \dots, r_d \in \N$. The output should include
- the Tucker decomposition produced by the algorithm,
- the vectors of singular values of the matrices that are explicitly approximated within the algorithm (one vector
of singular values per step)
- the Frobenius norms of the errors of the mentioned low-rank matrix approximation (one scalar per step).

In [147]:
using LinearAlgebra

function HOSVD(A::AbstractArray, r::NTuple{N,Integer}) where {N}
    # check arguments
    d = ndims(A)
    @assert d == N "Required number of rank parameters for given tensor is $d."
    @assert all(r .> 0) "Rank parameters need to be positive integers."

    # initialization
    n = size(A)
    r = min.(n, r) # ranks should not exceed dimensions
    Vs = Vector{AbstractMatrix}(undef, d)
    S = A

    for k = 1:d
        # unfolding matrix
        p = prod(r[1:k-1])
        q = prod(n[k+1:end])
        B = reshape(S, p, n[k], q)
        B = permutedims(B, [2,1,3])
        B = reshape(B, n[k], p*q)

        # truncated SVD
        V, Σ, U = svd(B)
        Vs[k] = V[:,1:r[k]]
        W = U'[1:r[k],:]
        for i = 1:r[k]
            W[i,:] *= Σ[i]
        end

        # next iterate
        S = reshape(W, r[k], p, q)
        S = permutedims(S, [2,1,3])
        S = reshape(S, r[1:k]..., n[k+1:end]...)
    end

    return S, Vs
end

HOSVD (generic function with 1 method)

### 2 Testing the HOSVD algorithm

Test your implementation of the HOSVD algorithm in the case of $d = 4$ as follows. For $n_k = 20 + k$ and $r_k = 2k$ with $k \in \{1, \dots, d\}$, generate a quasi-random Tucker decomposition $U_1, \dots, U_d, S$ of mode sizes $n_1, \dots, n_d$ and ranks $r_1, \dots, r_d$ by drawing the entries of the factors quasi-randomly in $[−1, 1]$. Form the tensor $C$ represented by the decomposition. Then compute a Tucker decomposition of $C$ with ranks $r_1, \dots, r_d$ using your implementation of the HOSVD algorithm. Form the tensor $\hat{C}$ represented by the output decomposition and compute the relative error $\frac{\|\hat{C} - C\|_F}{\|C\|_F}$.

In [148]:
"""mode contraction as implemented by Prof. Kazeev."""
function modecontract(S::AbstractArray, Z::AbstractMatrix, k::Integer)

    # check inputs
    d = ndims(S)
    n = collect(size(S))
    r, nk = size(Z)
    @assert 1 <= k <= d "Contraction index is not in range 1 ≤ k ≤ $d: got $k"
    @assert nk == n[k] "Dimension of matrix does not match tensor: expected $(n[k]), got $nk"
    
    # reshaping dimensions
    p = prod(n[1:k-1])
    q = prod(n[k+1:end])

    # reshaping and permuting
    S = reshape(S, p, n[k], q)
    S = permutedims(S, [2,1,3])
    S = reshape(S, n[k], p*q)

    # contraction
    S = Z * S
    
    # reshaping and permuting back
    n[k] = size(S, 1)
    S = reshape(S, n[k], p, q)
    S = permutedims(S, [2,1,3])
    S = reshape(S, n...)

    return S
end

modecontract

In [174]:
function evalTucker(S::AbstractArray, Vs::Vector{V}) where (V <: AbstractMatrix)
    # check arguments
    d = ndims(S)
    @assert length(Vs) == d "Required number of frame matrices is $d, got $(length(Vs))."
    r = size(S)
    T = eltype(S)
    n = Vector{Int}(undef, d)
    for k = 1:d
        nk, rk = size(Vs[k])
        @assert rk == r[k] "Dimension of frame matrix $k does not match Tucker core."
        n[k] = nk
        T = promote_type(T, eltype(Vs[k]))
    end

    # mode contraction
    A = S
    for k = 1:d
        A = modecontract(A, Vs[k], k)
    end

    return A
end

evalTucker (generic function with 2 methods)

In [182]:
d = 4
n = Tuple((21):(20+d))
r = Tuple(2*(1:d))

# construct C
Us = [rand(n[k], r[k]) for k = 1:d]
S = rand(r...)
C = evalTucker(S, Us)

# construct C hat
S, Us = HOSVD(C, r)
Ch = evalTucker(S, Us)

norm(Ch - C)/norm(C)

9.483792535282496e-16